<img src= "https://datascientest.fr/train/assets/logo_datascientest.png" style="height:150px"> 
<hr style="border-width:2px;border-color:#75DFC1">
<h1 style = "text-align:center" > Image processing </h1> 
<h2 style = "text-align:center"> Comment effectuer des transformations avancées sur une image </h2> 
<hr style="border-width:2px;border-color:#75DFC1">

### Contexte et objectif

> Nous avons déjà vu dans les notebooks précédents comment lire, modifier et filtrer facilement une image avec la librairie **`cv2`** (OpenCV). Nous pouvons aller encore plus loin en explorant des méthodes d'analyse d'image plus avancées permettant notamment d'améliorer potentiellement les performances d'un algorithme de classification.
>
> L'objectif de ce notebook est de découvrir des algorithmes de détection de formes et des méthodes de transformation morphologique comme l'érosion et la dilatation.
>
> Commençons par la phase d'importation des packages.

* Importer les packages **`numpy`**, **`cv2`** et **`pyplot`**

> Dans le notebook précédent nous avons vu qu'il était possible de supprimer du bruit dans une image comme les bruits poivre et sel ou gaussien, et également par des techniques de seuil changer la perspective de l'image. 
>
> Nous pouvons aller plus loin, pour ne pas simplement corriger des bruits éventuels, mais aussi détecter des formes dans l'image. Nous allons utiliser les fonctions **`GaussianBlur`** et **`Canny`** de **`cv2`**. Il est important dans un premier temps d'éliminer le bruit gaussien qui peut être présent pour obtenir ensuite une meilleure détection de formes. 
>
> Comme nous l'avons vu la fonction **`GaussianBlur`** fonctionne comme un filtre classique avec l'utilisation d'un noyau de convolution de dimension renseignée au préalable.
>
> La fonction **`Canny`** de **`cv2`** est l'implémentation de l'algorithme de John Canny développé en 1986. Il a été conçu pour détecter dans une image les contours caractéristiques. Pour chaque pixel ce dernier calcule l'intensité du contour selon une méthode spécifique. Ensuite pour chaque pixel l'algorithme décide si oui ou non l'intensité est suffisament forte pour considérer le pixel comme un contour important. Cette décision est prise à l'aide de deux seuils préalablement définis : 
* Si l'intensité est inférieure au seuil minimal, le pixel est rejeté
* Si l'intensité est supérieure au seuil maximal, le pixel est accepté comme contour
* Si l'intensité est entre les deux seuils, le pixel est accepté comme contour s'il est connecté à un pixel déjà accepté
>
> Cette procédure de sélection est ce qu'on appelle un seuillage à hystérésis. Vous trouverez des précisions sur l'algorithme **`Canny`** [ici]('https://fr.wikipedia.org/wiki/Filtre_de_Canny').
>
> <span style="color:#09b038; text-decoration : underline"> Exemple : utilisation de **GaussianBlur** et **Canny** </span><br>
```python 
    filtre = cv2.GaussianBlur(mon_image,(3,3),0) #pour supprimer le bruit gaussien dans mon_image
    edges = cv2.Canny(filtre,100,200) #pour détecter les formes de mon_image```


* Dans un premier temps, lire en couleur, l'image *building* qui est au format *.jpg*, la stocker dans une variable nommée **`building_color`**
* En utilisant la fonction **`GaussianBlur`** de **`cv2`**, éliminer le bruit gaussien dans **`building`**
* En utilisant la fonction **`Canny`** de **`cv2`**, détecter les formes de **`building`** en fixant les arguments **minVal** et **maxVal** respectivement à 100 et 200
* Afficher l'image filtrée

> Une fois l'image filtrée à l'aide de l'algorithme de **`Canny`**, on peut utiliser la fonction **`HoughLinesP`** pour déterminer les coordonnées des lignes pertinentes de l'image. Cette fonction renvoie un array  Si vous souhaitez avoir plus de détails sur la théorie mathématique derrière cette fonction, vous trouverez des informations [ici](https://docs.opencv.org/2.4/doc/tutorials/imgproc/imgtrans/hough_lines/hough_lines.html). 
> 
> Différents arguments sont utilisés dans la fonction : 
> * **dst**: Sortie du détecteur de formes. Il devrait s'agir d'une image en niveaux de gris (bien qu'il s'agisse en fait d'une image binaire)
> * **lignes** : Un vecteur qui va stocker les paramètres (x_{début}, y_{début}, x_{fin}, y_{fin}) des lignes détectées
> * **rho** : La résolution du paramètre r en pixels.
> * **theta** : La résolution du paramètre \theta en radians.
> * **threshold**: Le nombre minimum d'intersections pour "détecter" une ligne
> * **minLinLength**: Le nombre minimum de points qui peuvent former une ligne. Les lignes ayant moins que ce nombre de points ne sont pas prises en compte.
> * **maxLineGap**: L'écart maximum entre deux points à considérer dans une même ligne.
>
> <span style="color:#09b038; text-decoration : underline"> Exemple : utilisation de **HoughLinesP**</span><br>
```python 
   lines = cv2.HoughLinesP(edges,rho=3,theta=np.pi / 20,threshold=100,minLineLength=0,maxLineGap= 20)```
   
* Appliquer la fonction **`HoughLinesP`** à **`edges`** et stocker le résultat dans une variable **`lines`**.

> Une fois que nous avons stocker les lignes pertinentes nous allons les afficher en rouge dans l'image. Pour cela nous avons besoin dans un premier temps de créer un tableau rempli de 0 de même dimension que l'image **`building_color`** que nous appellerons **`line_img`**. 
>
> La fonction **`line`** de **`cv2`** permet de tracer les lignes dans l'image **`line_img`**. Il suffit de se  reservir des coordonnées stockées dans la variable **`lines`**. Elle contient pour chaque ligne les coordonnées du point de départ et du point de fin. L'argument **color** permet de donner une couleur spécifique aux lignes que nous souhaitons mettre en valeur dans l'image. L'argument **thickness** permet de gérer l'épaisseur des lignes.
> 
> <span style="color:#09b038; text-decoration : underline"> Exemple : utilisation de **lines**</span><br>
```python 
   for line in lines:
    for x1, y1, x2, y2 in line:
        cv2.line(mon_image, (x1, y1), (x2, y2), color = [255, 0, 0], thickness=3)```
>
> Une fois que nous avons que nous avons défini les lignes dans l'image **`line_img`** il suffit de 'concaténer' les deux images **`line_img`** et **`building_color`** pour faire apparaître ces lignes dans l'image d'origine. Pour cela nous allons utiliser la fonction **`addWeighted`**. Elle permet de calculer la somme pondérée de deux images.
>
> Pour utiliser la fonction il faut mettre dans l'ordre la première image, le poids associé, la seconde image, le poids associé et enfin un scalaire à ajouter en plus qu'on mettre à 0.
>
> <span style="color:#09b038; text-decoration : underline"> Exemple : utilisation de **addWeighted**</span><br>
```python 
  cv2.addWeighted(mon_image_1, 0.8, mon_image_2, 1.0, 0.0)```

* Créer **`line_img`** un tableau composé de zeros avec les mêmes dimensions que **`building_color`**.
* En utilisant la fonction **line**, déterminer en rouge les lignes détectées précedemment dans **`line_img`**.
* En utilisant la fonction **`addWeighted`**, sommer les images **`line_img`** et **`building_color`** et stocker le résultat dans une variable nommée **`building_lines`**.
* Afficher le résultat.

> On peut également utiliser des filtres avancés pour détecter des formes dans l'image. Le filtre de Sobel permet notamment de détecter les bords verticaux ou horizontaux d'une image. Comme pour un filtre classique le filtre Sobel utilise un noyau de convolution particulier qui lui permet de détecter les formes horizontales ou verticales dans l'image. 
>
> OpenCV met à disposition une fonction **`Sobel`** qui permet d'appliquer facilement le filtre à une image. Les arguments **dx** et **dy** permettent de déterminer si on souhaite mettre en évidence les bords verticaux ou horizontaux de l'image. Par exemple la combinaison (dx, dy) = (1,0) mettra en évidence les bords verticaux de l'image. L'inverse vous donnera les bords horizontaux. L'argument **ddepth** doit être égal à **cv2.CV_64F** pour une question technique liée aux types d'entiers utilisés.
>
> Suivant les combinaisons **dx**, **dy** ((1,0) ou (0,1)) le noyau de convolution utilisé n'est pas le même. Voici les deux noyaux utilisés suivant les cas :
>
><img src="https://assets-datascientest.s3-eu-west-1.amazonaws.com/notebooks/sobel_filters.png" style="height:300px">
>
> Le noyau de gauche est utilisé pour mettre en évidence les bords verticaux de l'image tandis que celui de droite est utilisé pour mettre en évidence les bords horizontaux. 
>
> Le filtre s'utilise sur des images en **niveaux de gris**.
>
> <span style="color:#09b038; text-decoration : underline"> Exemple : utilisation de **Sobel** </span><br>
```python 
    sobel = cv2.Sobel(mon_image, ddepth = cv2.CV_64F, dx = 1, dy = 0) #pour détecter les bords verticaux dans mon_image```

* Dans un premier temps, lire en niveaux de gris, l'image building qui est au format .jpg, la stocker dans une variable nommée **`street_gray`**
* En utilisant **`Sobel`**, filtrer **`street_gray`** de sorte à faire apparaître les bords verticaux à l'intérieur de l'image

> On constate que le filtre a bien détecté les bords verticaux dans l'image. Ce type de filtre est utile pour mieux comprendre les structures présentes dans une image comme une maison par exemple. 
>
> On peut aller un peu plus loin avec des filtres plus avancés comme le **`Laplacian`**. Il utilise la mise en évidence des discontinuités de niveaux de gris dans une image et tente de désaccentuer les régions dont les niveaux de gris varient lentement. Le résultat de cette opération est la production d'images dont les bords sont grisâtres et qui présentent d'autres discontinuités sur un fond sombre. Cela produit des bords intérieurs et extérieurs dans une image.
>
> Une différence entre le **`Laplacian`** et les autres filtres comme **`Sobel`** est que, contrairement aux autres opérateurs, le laplacien ne prend pas de bords dans une direction particulière comme vertical ou horizontal. 
>
>  L'argument **ddepth** doit être égal à **cv2.CV_64F** pour une question technique liée aux types d'entiers utilisés.
>
> <span style="color:#09b038; text-decoration : underline"> Exemple : utilisation de **Laplacian** </span><br>
```python 
    laplacian = cv2.Laplacian(mon_image, ddepth = cv2.CV_64F)```
    
* En utilisant la fonction **`Laplacian`**, filtrer **`street_gray`** et observer le résultat

> Le filtrage paraît plus affiné qu'avec **Sobel** et permet de distinguer des structures plus complexes au sein de l'image. 
>
> Ces filtres sont déterminants dans la lecture d'images pour déterminer des caractéristiques clés. Des algorithmes complexes comme les réseaux de neurones CNN utilisent un très grand nombre de filtres pour extraire au fur et à mesure des informations clés de plus en plus complexes leur permettant d'apprendre de manière efficace et par la suite faire de puissantes prédictions.
>
> On peut également utiliser ce qu'on appelle des méthodes de transformations morphologiques pour détecter des structures pertinentes et éliminer celles non pertinentes pour la compréhension de l'image. Les deux méthodes de base très utilisées sont la dilatation et l'érosion. 
>
> Une première transformation morphologique simple à appliquer est l'érosion. Elle est utilisée pour faire disparaître les limites des objets de premier plan et également pour éliminer les petits bruits blancs des images. L'érosion peut également être utilisée pour détacher deux images reliées entre elles. 
>
> Elle s'utilise facilement avec la fonction **`erode`** de **`cv2`**. Il est nécessaire avant de définir un noyau de convolution qui peut avoir n'importe quelle forme ou taille. On définit le point d'ancrage du noyau comme le centre du noyau.
>
> Lorsque le noyau balaie l'image, nous calculons la valeur minimale de la fenêtre recouverte par le noyau et nous remplaçons le point d'ancrage par cette valeur. 
>
> L'érosion a pour conséquence d'affiner l'image et notament les zones claires. Pour comprendre et visualiser les effets de l'érosion nous allons utiliser **`erode`** sur une image d'araignée.
>
> <span style="color:#09b038; text-decoration : underline"> Exemple : utilisation de **erode** </span><br>
```python 
    kernel = np.ones((3,3),np.uint8) # création du noyau
    erosion = cv2.erode(mon_image, kernel) # erosion de mon_image```
    

* Lire, en niveaux de gris, l'image *spider* au format *.jpeg* et la stocker dans une variable nommée **`spider_gray`**
* Redimensionner l'image en taille 150x150 à l'aide de **`resize`**
* Inverser les nuances de gris pour mettre en blanc les formes au premier plan
* Filtrer l'image avec un filtre gaussien et la stocker dans une variable nommée **`filtre`**
* Effectuer 3 érosions à l'aide de la fonction **`erode`** de **`cv2`** en utilisant 3 noyaux de convolutions composés uniquement de 1 mais de dimensions respectives (3,3), (5,5) et (7,7) 
* Afficher le résultat pour chaque érosion ainsi que l'image d'origine

> L'érosion permet d'isoler le corps de l'araignée et d'éliminer la présence des pattes. C'est très utile pour isoler des parties spécifiques que l'on souhaite identifier et étudier. 
>
> Une autre transformation morphologique très connue est la dilatation. Elle produit l'effet inverse de l'érosion et fonctionne de manière identique. 
>
> Il est nécessaire avant de définir un noyau de convolution qui peut avoir n'importe quelle forme ou taille. On définit le point d'ancrage du noyau comme le centre du noyau. Lorsque le noyau balaie l'image, nous calculons la valeur maximale de la fenêtre recouverte par le noyau et nous remplaçons le point d'ancrage par cette valeur.
>
> La dilatation est facilement utilisable avec la fonction **`dilate`** de **`cv2`**. 
>
> <span style="color:#09b038; text-decoration : underline"> Exemple : utilisation de **dilate** </span><br>
```python 
    kernel = np.ones((3,3),np.uint8) # création du noyau
    dilatation = cv2.dilate(mon_image, kernel) # erosion de mon_image```
    
* Effectuer une dilatation de l'image **`erosion_2`** à l'aide d'un noyau de convolution composé de 1 de taille (5,5)
* Afficher l'image dilatée

* Effectuer la différence entre l'image d'origine **`spider_gray`** et l'image dilatée **`dilation`**
* Afficher le résultat

> On peut voir qu'en dilatant le corps de l'araignée et en soustrayant à l'image d'origine, on obtient les contours de l'araignée. Ces techniques de transformations morphologiques permettent de détecter ou d'isoler des structures pertinentes au sein d'une image.

### Conclusion

>Dans ce notebook vous aurez appris à :
>
>* Détecter des formes complexes en utilisant des algorithmes avancés comme **`Canny`**.
>* Effectuer des filtrages avancés comme **`Sobel`** ou **`Laplacian`** pour détecter des formes verticales ou horizontales dans une image. 
>* Effectuer des transformations morphologiques simples pour éroder ou dilater une image.